Bibliography:

* [Stacked Convolutional Auto-Encoders for Hierarchical Feature Extraction](http://people.idsia.ch/~ciresan/data/icann2011.pdf)
    

In [1]:
import torch
import torchvision
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from torchvision import datasets
from torchvision.utils import save_image

import skimage 
import math
# import io
# import requests
# from PIL import Image

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import sys
import os


In [2]:
import cae

In [3]:
from helpers import *
from helper_modules import *
from multi_res_cae import *

In [4]:
%load_ext autoreload
%autoreload

In [5]:
%aimport helpers, helper_modules, multi_res_cae

In [6]:
%aimport

Modules to reload:
helper_modules helpers multi_res_cae

Modules to skip:



In [7]:
# frcae = MultiFullCAE((640,480))

In [8]:
# mrcae = MultiResCAE([640,480])

In [9]:
# Hyper Parameters
# num_epochs = 5
# batch_size = 100
# learning_rate = 0.001

num_epochs = 20
batch_size = 128
learning_rate = 0.0001

In [10]:
%%time

#%time model = MultiFullCAE(in_img_shape=(32,32), full_image_resize=(24,24)).cuda()
model = MultiResCAE(in_img_shape=[32,32], channels=3, conv_layer_feat=[16, 32, 64],
                 res_px=[[24, 24], [16, 16], [12, 12]], crop_sizes=[[32, 32], [24,24], [12, 12]],
                 # conv_sizes = [(3,5,7), (3,5,7,11), (3,5,7,11)]  # this is too much I think
                 # conv_sizes=[[1, 3, 5], [1, 3, 5], [1, 3, 5, 7]]  # test b
#                  conv_sizes=[[5, 7, 11], [3, 5, 7, 9], [1, 3, 5]]  # test c
                 conv_sizes=[[5, 7], [3, 5, 7], [1, 3, 5]]  # test d
        ).cuda()

CPU times: user 1.29 s, sys: 330 ms, total: 1.62 s
Wall time: 1.62 s


In [11]:
# model.parameters

In [12]:
%%time
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

CPU times: user 937 µs, sys: 0 ns, total: 937 µs
Wall time: 940 µs


In [13]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 32, 32)
    return x


In [14]:
%%time

#transformation = monochrome_preprocess(32,32)
transformation = fullimage_preprocess(32,32)
#train_loader, test_loader = get_loaders(batch_size, transformation, dataset=datasets.CocoDetection)
train_loader, test_loader = get_loaders(batch_size, transformation)

Files already downloaded and verified
CPU times: user 681 ms, sys: 212 ms, total: 892 ms
Wall time: 891 ms


In [15]:
%%time

for epoch in range(num_epochs):
    for i, (img, labels) in enumerate(train_loader):
        img = Variable(img).cuda()
        # ===================forward=====================
#         print("encoding batch of  images")
        output = model(img)
#         print("computing loss")
        loss = criterion(output, img)
        # ===================backward====================
#         print("Backward ")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data[0]))
    if epoch % 4 == 0:
        pic = to_img(output.cpu().data)
        in_pic = to_img(img.cpu().data)
        save_image(pic, './mrcae_results/e_in-32x32_1-3-5_7-out_image_{}.png'.format(epoch))
        save_image(in_pic, './mrcae_results/e_in-32x32_1-3-5_7-in_image_{}.png'.format(epoch))
#     if loss.data[0] < 0.35: #arbitrary number because I saw that it works well enough
#         break

epoch [1/20], loss:0.6429


/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


epoch [2/20], loss:0.4522
epoch [3/20], loss:0.4150
epoch [4/20], loss:0.4652
epoch [5/20], loss:0.4166
epoch [6/20], loss:0.4078


Process Process-7:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [16]:
#torch.save("fmrcae_in-64x64_32x32_3-5-7-11.pth", model)
#torch.save("mrcae_in-32x32_.pth", model)